# GPUの種類

Colabがあなたに割り当てるGPUの種類は、トレーニング時間に大きく影響します。Colabで達成したいくつかのサンプルタイムをここに示す。 Colab Proは通常V100でスタートしますが、24時間ノンストップでスクリプトを実行し続けた場合、P100に戻される。

* 1024x1024 - V100 - 566秒/tick(CoLab Pro)
* 1024x1024 - P100 - 1819秒/tick(CoLab Pro)
* 1024x1024 - T4 - 2188 sec/tick (CoLab Free)

Google CoLab Proを使用する場合、一般的に、非アクティブでも、24時間以内に切断されることはない。 無料のCoLabでは、数時間操作をしないと、完全に動作しているスクリプトが切断される。 この問題を回避する方法は、以下の通り。

* Google Colabが切断されないようにするには](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

# 環境構築

24時間以上のトレーニングが必要。 Colabは数時間おきにセッションを切断する。 このとき、トレーニングを再開できるように準備しておく必要がある。 トレーニングはtickに分けられ、多くのtick（デフォルトでは50）ごとにニューラルネットワークが評価され、スナップショットが保存される。 Colabがシャットダウンすると、最後のスナップショット以降のトレーニングはすべて失われる。1tickごとにスナップショットするのが望ましいと思われるが、このスナップショット処理自体に1時間近くかかってしまう。 解像度や学習データに応じて最適なスナップショットサイズを知ることが重要です。

GDRIVEをマウントし、スナップショットをそこに保存するようにします。 また、トレーニング用の画像もGDRIVEに配置する必要があります。

NVIDIA StyleGAN2 ADA PyTorchもインストールする必要がある。 また、PyTorchをStyleGANをサポートするバージョンにダウングレードする必要がある。

In [ ]:
!pip uninstall tensorboard
!pip install tensorboard==1.14.1
!pip uninstall jax jaxlib
!pip install jax[cpu]==0.3.10

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

In [ ]:
import os
os.chdir("/content")
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
!pip install icrawler

# 画像収集 & 画像変換

Googleの検索機能を用いて画像を収集し、適切なサイズの画像へと修正する。

In [ ]:
gender = 'Men' #@param ["Men","Women"]{allow-input: false}

In [ ]:
from icrawler.builtin import GoogleImageCrawler
import os

if not os.path.exists('/content/drive/MyDrive/data/gan/images'):
  os.makedirs('/content/drive/MyDrive/data/gan/images')
  os.makedirs('/content/drive/MyDrive/data/gan/dataset')
  os.makedirs('/content/drive/MyDrive/data/gan/experiments')

os.chdir('/content/drive/MyDrive/data/gan/images')
print(os.getcwd())

if not (os.path.exists('./drive/MyDrive/data/gan/images/'+ gender +'/000000')):
  crawler = GoogleImageCrawler(storage={"root_dir": gender+"/000000"})
  crawler.crawl(keyword="Japanese Face " + gender, max_num = 200)

In [ ]:
import os
import glob
from PIL import Image

dir = "/content/drive/MyDrive/data/gan/images/"+ gender +"/000000/*"

list = glob.glob(dir)
i = 1

for file in list:
    new_file_name =('/content/drive/MyDrive/data/gan/images/' + gender + '/000000/' + str(i).zfill(6) + '_trimmed.jpg')
    img = Image.open(file)
    resized_img = img.resize((256,256))
    resized_img.convert('RGB').save(new_file_name)
    os.remove(file)
    i+=1

In [ ]:
source = "/content/drive/MyDrive/data/gan/images/" + str(gender) + "/000000"
dataset = "/content/drive/MyDrive/data/gan/dataset/" + str(gender)

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source=$source --dest=$dataset

新しく作成したデータセットを消去するには、次のコマンドを実行する。 何か問題が起きて、画像を整理して上記のコマンドを再実行する必要がある場合は、部分的に作成されたデータセットディレクトリを削除しておく必要がある。

In [ ]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/Men/*

# 画像の一括調整

すべての画像の寸法と色深度が同じであることが重要。 このコードにより、問題のある画像を特定することができる。

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images/' + gender + "/000000"
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


# 初期の実行
初期以外でこのセルを実行してはいけない。

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset/"+gender
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

# 実行の再開

In [ ]:
import os
os.chdir("/content")
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

In [ ]:
import os

# 要望に合わせて修正する
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments/"+gender
snapshot-version = 160 #@param {type:"number"}
NETWORK = "network-snapshot-"+str(snapshot-version.zfill(6))+".pkl"
experiment-directory = "00000-Men-auto1" #@param {type:"string"}
RESUME = os.path.join(EXPERIMENTS, experiment-directory, NETWORK)
useDataset = False #@param {type:"boolean"}
DATA;
if (useDataset){
    dataset-version = 0 #@param {type:"number",min:0,max:999999}
    DATA = "/content/drive/MyDrive/data/gan/dataset/"+gender+"/"+str(dataset-version.zfill(5))
} else {
    rate = 0 #@param {type:"slider",min:0,max:5,step:1}
    DATA = "/content/drive/MyDrive/data/gan/rated_images/"+gender+"/"+rate
}

snap = 10 #@param {type:"number"}

# コマンドを実行
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {snap} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

# 生成画像の編集

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import glob
import os

files = glob.glob('/content/drive/MyDrive/data/gan/experiments/'+gender+'/*/fakes*.png')

for file in files:
  if gender in file:
    #画像の読み込み
    img=cv2.imread(file,cv2.IMREAD_COLOR)
    h,w=img.shape[:2]
    split_x=30
    split_y=16
    #画像の分割処理
    cx=0
    cy=0
    os.makedirs('/content/drive/MyDrive/data/gan/split_pic/'+gender+'/gen_'+str(files.index(file)))
    for i in rnage(6):
      os.makedirs('/content/drive/MyDrive/data/gan/rated_images/'+gender+"/"+str(i))
    for j in range(split_x):
        for i in range(split_y):
            split_pic=img[cy:cy+int(h/split_y),cx:cx+int(w/split_x),:]
            cv2.imwrite('/content/drive/MyDrive/data/gan/split_pic/'+gender+'/gen_'+str(files.index(file))+'/split_y'+str(i)+'_x'+str(j)+'.png',split_pic)
            cy=cy+int(h/split_y)
        cy=0
        cx=cx+int(w/split_x)